In [1]:
import holoviews as hv
import torch
import numpy as np
import dgl
import dgl.nn as dglnn
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch
import torch.nn as nn
import numpy as np
import sys
import panel as pn
pn.extension()
hv.extension('bokeh')

sys.path.append('../../..')
from gnnfairviz import eug
sys.path.remove('../../..')

sys.path.append('../..')
from models.sage import SAGE, get_embeddings
from datasets import Nba
from util import preprocess_nba, train
sys.path.remove('../..')

nba = Nba()
g, adj, features, sens, sens_names, masks, labels, feat_names = preprocess_nba(nba)

# create SAGE model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
in_size = features.shape[1]
out_size = int(sum(labels.unique() != -1))
model = SAGE(in_size, 16, out_size).to(device)

# training

In [2]:
train(g, features, labels, masks, model, save_path='demo_nba_sage.pth')

Epochs:   1%|          | 13/2000 [00:01<02:51, 11.57it/s]

Epoch 00000 | Loss 1.0753 | Val Accuracy 0.4883 | Val F1 0.4608


Epochs:   6%|▌         | 112/2000 [00:02<00:19, 94.52it/s]

Epoch 00100 | Loss 0.6478 | Val Accuracy 0.7136 | Val F1 0.7045


Epochs:  11%|█         | 222/2000 [00:03<00:16, 105.93it/s]

...

Epoch 01400 | Loss 0.5488 | Val Accuracy 0.7230 | Val F1 0.7208


Epochs:  76%|███████▌  | 1521/2000 [00:15<00:04, 107.27it/s]

Epoch 01500 | Loss 0.5560 | Val Accuracy 0.7230 | Val F1 0.7208


Epochs:  81%|████████  | 1616/2000 [00:16<00:03, 97.59it/s] 

Epoch 01600 | Loss 0.5856 | Val Accuracy 0.7230 | Val F1 0.7208
Early stopping triggered.


In [2]:
# load the model
model.load_state_dict(torch.load('demo_nba_sage.pth'))
model.eval()

SAGE(
  (layers): ModuleList(
    (0): SAGEConv(
      (feat_drop): Dropout(p=0.0, inplace=False)
      (fc_neigh): Linear(in_features=94, out_features=16, bias=False)
    )
    (1): SAGEConv(
      (feat_drop): Dropout(p=0.0, inplace=False)
      (fc_neigh): Linear(in_features=16, out_features=16, bias=False)
    )
  )
  (mlp): Linear(in_features=16, out_features=2, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

# fairness analysis

In [3]:
e = eug.EUG(model, adj, features, sens, sens_names, masks, labels, get_embeddings, feat_names=feat_names)
app = e.show()
server = pn.serve(app, port=45788)

/home/newdisk/xinwu/anaconda3/envs/holoviews_clone/lib/python3.11/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


/home/newdisk/xinwu/anaconda3/envs/holoviews_clone/lib/python3.11/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")
100%|██████████| 94/94 [00:00<00:00, 260.88it/s]


tensor([1.0000, 0.2159, 0.0581,  ..., 0.0790, 0.1917, 1.0000], device='cuda:0')
nnz 21242


100%|██████████| 1/1 [00:00<00:00, 5071.71it/s]


Launching server at http://localhost:45788


100%|██████████| 94/94 [00:03<00:00, 26.45it/s]


In [4]:
server.stop()

In [9]:
records = e.get_records()

In [10]:
import pickle
import numpy as np

with open('demo_nba_sage_records.pkl', 'wb') as f:
    pickle.dump(records, f)

In [3]:
import pickle

# Open the file in binary read mode
with open('demo_nba_sage_records.pkl', 'rb') as f:
    records = pickle.load(f)

In [4]:
records

[{'Sens': ['Country', 'Age'],
  'Nodes': array([  7,  15,  18,  52,  69,  80,  81, 100, 124, 170, 173, 188, 192,
         226, 281, 289, 308, 329, 331, 342, 349, 363, 367, 374, 383, 389]),
  'Edges': True,
  'Attributes': []},
 {'Sens': ['Country', 'Age'],
  'Nodes': array([  6,   7,   8,  13,  15,  17,  18,  24,  28,  38,  39,  41,  48,
          51,  57,  63,  66,  68,  71,  73,  80,  81,  82,  83,  84,  85,
          86,  87,  88,  95,  98, 100, 105, 106, 107, 109, 110, 112, 113,
         117, 120, 123, 124, 125, 129, 132, 138, 140, 142, 143, 145, 147,
         152, 153, 158, 159, 161, 163, 166, 167, 170, 171, 172, 173, 174,
         175, 177, 184, 189, 191, 193, 195, 198, 201, 207, 208, 209, 224,
         226, 228, 232, 235, 242, 245, 246, 249, 251, 253, 255, 257, 267,
         268, 270, 272, 275, 278, 281, 282, 283, 284, 286, 287, 288, 289,
         291, 292, 295, 296, 303, 304, 305, 308, 309, 311, 312, 317, 319,
         320, 321, 322, 323, 326, 328, 329, 335, 336, 339, 341, 342,

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
feat = features
feat_clone = feat.clone().detach()
adj_clone = adj.clone().detach().to(device)

## attr perturbation

In [6]:
selected_nodes = records[1]['Nodes']
attr_indices = []
for attr in records[1]['Attributes']:
    attr_indices.append(feat_names.index(attr))
feat_mean = feat.mean(dim=tuple(range(feat.dim() - 1)))
row_indices = torch.tensor(selected_nodes)  
col_indices = torch.tensor(attr_indices) 
# Convert row and column indices to a meshgrid of indices
rows, cols = torch.meshgrid(row_indices, col_indices, indexing='ij')
# Use fancy indexing to set the specified elements to 1
feat_clone[rows, cols] = feat_mean[attr_indices] 

## structure perturbation

In [7]:
selected_nodes = records[0]['Nodes']
# Converting the rows_to_zero into a tensor
selected_nodes_tensor = torch.from_numpy(selected_nodes.astype(int)).to(device)

# Get the row indices and column indices separately from the adj tensor
indices = adj_clone.coalesce().indices()
row_indices = indices[0]
col_indices = indices[1]

# Mask to zero out the selected rows
row_mask = ~torch.isin(row_indices, selected_nodes_tensor)
col_mask = ~torch.isin(col_indices, selected_nodes_tensor)
mask = row_mask & col_mask

# Apply mask
new_row_indices = row_indices[mask]
new_col_indices = col_indices[mask]
# new_values = adj_clone.values()[mask]

# Determine which diagonal indices need to be added
needed_diagonals = selected_nodes_tensor
existing_diagonals = (new_row_indices == new_col_indices) & torch.isin(new_row_indices, needed_diagonals)
missing_diagonals = needed_diagonals[~torch.isin(needed_diagonals, new_row_indices[existing_diagonals])]

# Add missing diagonal elements
if missing_diagonals.numel() > 0:
    new_row_indices = torch.cat([new_row_indices, missing_diagonals])
    new_col_indices = torch.cat([new_col_indices, missing_diagonals])
    # new_values = torch.cat([new_values, torch.ones_like(missing_diagonals, dtype=torch.float32)])

g_new = dgl.heterograph({('node', 'edge', 'node'): (new_row_indices.cpu().numpy(), new_col_indices.cpu().numpy())}) 
g_new = g_new.int().to(feat.device)

## retrain

In [36]:
model_new = SAGE(in_size, 16, out_size).to(device)
train(g_new, feat_clone, labels, masks, model_new, save_path='demo_nba_sage_retrain.pth')

Epochs:   0%|          | 0/2000 [00:00<?, ?it/s]

Epochs:   1%|          | 21/2000 [00:00<00:19, 103.93it/s]

Epoch 00000 | Loss 0.6977 | Val Accuracy 0.5258 | Val F1 0.3675


Epochs:   6%|▌         | 121/2000 [00:01<00:17, 104.79it/s]

Epoch 00100 | Loss 0.6133 | Val Accuracy 0.5915 | Val F1 0.5612


...

Epoch 01400 | Loss 0.5947 | Val Accuracy 0.6854 | Val F1 0.6794


Epochs:  76%|███████▌  | 1522/2000 [00:14<00:04, 106.03it/s]

Epoch 01500 | Loss 0.6147 | Val Accuracy 0.6854 | Val F1 0.6794


Epochs:  80%|███████▉  | 1596/2000 [00:14<00:03, 106.66it/s]

Early stopping triggered.


In [ ]:
# load the model
model_new.load_state_dict(torch.load('demo_nba_sage_retrain.pth'))
model_new.eval()

In [38]:
e_new = eug.EUG(model_new, adj, features, sens, sens_names, masks, labels, get_embeddings, feat_names=feat_names)
app_new = e_new.show()
server_new = pn.serve(app_new, port=45780)

/home/newdisk/xinwu/anaconda3/envs/holoviews_clone/lib/python3.11/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")
/home/newdisk/xinwu/anaconda3/envs/holoviews_clone/lib/python3.11/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")
100%|██████████| 94/94 [00:00<00:00, 256.43it/s]


tensor([1.0000, 0.2159, 0.0581,  ..., 0.0790, 0.1917, 1.0000], device='cuda:0')
nnz 21242


100%|██████████| 1/1 [00:00<00:00, 7332.70it/s]


Launching server at http://localhost:45780


100%|██████████| 94/94 [00:03<00:00, 24.63it/s]
